In [1]:
import cv2
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import numpy as np
from pathlib import Path

In [9]:
# 设置路径
root = Path("data/small")
ann = root / "annotation.json"
img_prefix = root / "images"
cat_name = "0"

In [17]:
# 初始化 coco 数据集的 API
coco = COCO(ann)
total_imgs = len(coco.dataset["images"])

for img_id in range(total_imgs):
    # 对于每张图片，根据 id 获得图片的路径
    img_name = coco.loadImgs(img_id)[0]["file_name"]
    path = img_prefix / img_name
    img = cv2.imread(str(path))  # 读取得到 3 维 numpy

    # 根据名称得到 category，在这里是前景叶子的 0
    cat_id = coco.getCatIds(catNms=[cat_name])

    # 根据图片 id 和 category id 获得所有标注 ann
    ann_ids = coco.getAnnIds(imgIds=img_id, catIds=cat_id)
    anns = coco.loadAnns(ann_ids)

    # 创建标签为 0 的区域的遮罩矩阵（二值化）
    mask = np.zeros(img.shape[0:2], dtype=np.int0)
    for ann in anns:
        mask = mask | (coco.annToMask(ann) > 0)  # bitwise_or
    covered_img = img * mask[:, :, np.newaxis]
    # plt.imshow(covered_img)  # 显示遮罩矩阵

    # save
    dest = root / "covered" / img_name
    dest.parent.mkdir(exist_ok=True)
    cv2.imwrite(str(dest), covered_img)